In [10]:
import torch
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from dgl.nn import GraphConv,SAGEConv
import dgl
from dgl.data import DGLDataset
import seaborn as sns
import random
from tqdm import tqdm
import sklearn
from sklearn.model_selection import train_test_split

In [9]:
class ICPKGIDataset(DGLDataset):
    def __init__(self,data_path,transforms=None):
        self.data_path = data_path
        self.transforms = transforms
        super().__init__(name='ICPKGI_gprah')
    
    def process(self):
        GRAPHS, LABELS = dgl.load_graphs(self.data_path) #保存したグラーフデータの読み込み
        self.graphs = GRAPHS #グラフリストを代入
        self.labels = LABELS['label'] #ラベル辞書の値のみ代入
        self.dim_nfeats=len(self.graphs[0].ndata['f'][0])

    def __getitem__(self, idx):
        if self.transforms == None:
            return self.graphs[idx], self.labels[idx]
        else:
            data=self.transforms(self.graphs[idx])
            return data,self.labels[idx]
    def __len__(self):
        return len(self.graphs)
    

class EmbeddingNetwork(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(EmbeddingNetwork, self).__init__()
        self.conv1 = SAGEConv(in_feats, hidden_feats, aggregator_type='mean')
        self.conv2 = SAGEConv(hidden_feats, hidden_feats, aggregator_type='mean')
        self.conv3 = SAGEConv(hidden_feats, out_feats, aggregator_type='mean')
        '''self.conv1 = GraphConv(in_feats, hidden_feats)
        self.conv2 = GraphConv(hidden_feats, hidden_feats)'''
        self.flatt=nn.Flatten()

    def forward(self, g, features):
        '''x=self.flatt(features)
        x = torch.relu(self.conv1(g, x))
        x = self.conv2(g, x)
        #x = self.conv3(g,x)'''
        
        x = self.flatt(features)
        x = torch.relu(self.conv1(g,x))
        x = torch.relu(self.conv2(g,x))
        x = self.conv3(g,x)

        g.ndata['h'] = x
        return g
    

def _train_test_split(data,data_num):
    shuffle_data=random.sample(data,len(data))
    return shuffle_data[:-data_num], shuffle_data[-data_num:]


In [6]:
graphs=[[] for _ in range(5)]
dataset=ICPKGIDataset('../data/ICPKGI/8patch_gray.dgl')


In [11]:
trainset,testset=train_test_split(dataset,test_size=0.2,shuffle=True,stratify=True)

InvalidParameterError: The 'stratify' parameter of train_test_split must be an array-like or None. Got True instead.